# Preprocess test and train dataset

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Wed Jun  3 18:25:19 2020

@author: orestis
"""

from pandas import DataFrame,read_csv

import pandas as pd
import nltk
import string

#Location of train data
TrainLocation = '/home/orestis/Desktop/GitHubRepositories/DataMining-MachineLearning/CommentsClassification/data/train.csv'
#Location of test data
TestLocation = '/home/orestis/Desktop/GitHubRepositories/DataMining-MachineLearning/CommentsClassification/data/test.csv'

train_df = pd.read_csv(TrainLocation,names=['Class','Date','Comment'],skiprows=1)   #Get the training data
test_df = pd.read_csv(TestLocation,names=['Class','Date','Comment'],skiprows=1)     #Get the testing data

#Delete date column
del train_df['Date']
del test_df['Date']

train_comments = train_df['Comment']
test_comments = test_df['Comment']

#PREPROCESS COMMENTS DATA
alphabet = list(string.ascii_lowercase)
numbers=['1','2','3','4','5','6','7','8','9','0',' ']
alphabet = alphabet + numbers
#Convert to lower case
for i in range(len(train_comments)):
    train_comments[i]=train_comments[i].lower()
    train_comments[i] = train_comments[i].replace('-',' ')
    train_comments[i] = train_comments[i].replace('\\',' ')
    train_comments[i] = train_comments[i].replace('.',' ')
    train_comments[i] = train_comments[i].replace(',',' ')
    train_comments[i] = train_comments[i].replace('_',' ')
    #Keep only the letters and numbers
    for c in train_comments[i]:
        if c not in alphabet:
             train_comments[i]=train_comments[i].replace(c,'')

for i in range(len(test_comments)):
    test_comments[i]=test_comments[i].lower()
    test_comments[i] = test_comments[i].replace('-',' ')
    test_comments[i] = test_comments[i].replace('\\',' ')
    test_comments[i] = test_comments[i].replace('.',' ')
    test_comments[i] = test_comments[i].replace(',',' ')
    test_comments[i] = test_comments[i].replace('_',' ')
    #Keep only the letters and numbers
    for c in test_comments[i]:
        if c not in alphabet:
             test_comments[i]=test_comments[i].replace(c,'')
    
train_class_values = train_df['Class']

#Create the new data sets
NewTrainDataSet = list(zip(train_comments,train_class_values))
NewTestDataSet = list(test_comments)

new_train_df = pd.DataFrame(data=NewTrainDataSet,columns=['Comments','Classs'])
new_test_df = pd.DataFrame(data=NewTestDataSet,columns=['Comments'])

new_train_df.to_csv('train.csv',index=False,header=False)
new_test_df.to_csv('test.csv',index=False,header=False)

# CountVectorizer and Naive Bayes Prediction

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  4 09:29:45 2020

@author: orestis
"""

from pandas import DataFrame,read_csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

import pandas as pd

#Function to get accuracy of the predicted results
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

TrainDataLocation = './train.csv'
TestDataLocation = './test.csv'

train_df = pd.read_csv(TrainDataLocation,header=None,names=['Comment','Class'])
test_df = pd.read_csv(TestDataLocation,header=None,names=['Comment','Class'])

train_comments = train_df['Comment']
test_comments = test_df['Comment']

train_classes=train_df['Class']
test_classes =test_df['Class']


#Create a Gaussian Naive Bayes model
model = GaussianNB()

#Create Train Count Vectorizer
TrainCount_Vectorizer = list()

vectorizer = CountVectorizer()
predicted_classes=list()
#Make the predictions
for test_comment in test_comments:
    TrainCount_Vectorizer=[]
    #tokenize and build vocabulary
    vectorizer.fit([test_comment])
    for x in train_comments:
        comment = [x]
        #encode the document
        vector = vectorizer.transform(comment)
        temp_array = vector.toarray()
        TrainCount_Vectorizer.append(temp_array[0])
    
    #Predict Output
    model.fit(TrainCount_Vectorizer,train_classes)
    vector = vectorizer.transform([test_comment])
    predicted = model.predict(vector.toarray())
    predicted_classes.append(predicted)

score = accuracy_metric(test_classes, predicted_classes)
print('Score:',score)

/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in

Score: 60.581655480984345


# Prediction After removing stopwords and lemmization

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  5 12:56:30 2020

@author: orestis
"""
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

DataLocation = './data.csv'

df = pd.read_csv(DataLocation,header=None,names=['Comment','Class'])

X_train,X_test,y_train,y_test = train_test_split(df['Comment'],df['Class'], random_state=1)

cv = CountVectorizer(strip_accents='ascii',token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b',lowercase=True,stop_words='english')

X_train_cv=cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_cv, y_train)
predictions = naive_bayes.predict(X_test_cv)

print('Accuracy score: ',accuracy_score(y_test, predictions))

Accuracy score:  0.7697283311772316


# Scores of NaiveBayes,Random Forest,SVM with the use of TF-IDF

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  5 12:56:30 2020

@author: orestis
"""
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

DataLocation = './data.csv'

df = pd.read_csv(DataLocation,header=None,names=['Comment','Class'])

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1,2), stop_words='english')

features = tfidf.fit_transform(df.Comment).toarray()
labels = df.Class


X_train,X_test,y_train,y_test = train_test_split(df['Comment'],df['Class'], random_state=0)

count_vect = CountVectorizer()

X_train_counts=count_vect.fit_transform(X_train)
X_test_cv = count_vect.transform(X_test)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

naive_bayes = MultinomialNB()
predictions = clf.predict(X_test_cv)

print('Naive Bayes score: ',accuracy_score(y_test, predictions))

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])


cv_df.groupby('model_name').accuracy.mean()

Naive Bayes score:  0.7289780077619664


model_name
LinearSVC                 0.764795
RandomForestClassifier    0.656098
Name: accuracy, dtype: float64

# Score of Naive Bayes after lemmazation removing stopwords,Laplace Smoothing,biagrams and TF-IDF

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  5 15:34:22 2020

@author: orestis
"""
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
import re

lemmatizer = WordNetLemmatizer() 
DataLocation = './data.csv'

first_df = pd.read_csv(DataLocation,header=None,names=['Comment','Class'])

comments_biagrams=list()
#Create a new list with the bigrams of the comments
for i in range(len(first_df['Comment'])):
    txt1 = first_df['Comment'][i]
    # initializing list  
    test_list = [txt1] 
      
    # using list comprehension + enumerate() + split() 
    # for Bigram formation 
    res = [(x, i.split()[j + 1]) for i in test_list  
           for j, x in enumerate(i.split()) if j < len(i.split()) - 1] 
    
    new_comment = ''
    for x in res:
        #Create a new comment with the lemmatized biagrams
        new_comment = new_comment + lemmatizer.lemmatize(x[0]) + lemmatizer.lemmatize(x[1]) + ' '
    comments_biagrams.append(new_comment)

#Create a new dataset with the biagrams
class_values = first_df['Class']
DataSet = list(zip(comments_biagrams,class_values))
df = pd.DataFrame(data=DataSet,columns=['Comment','Class'])

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1,2), stop_words='english')

features = tfidf.fit_transform(df.Comment).toarray()
labels = df.Class


X_train,X_test,y_train,y_test = train_test_split(df['Comment'],df['Class'], random_state=0)

count_vect = CountVectorizer()

X_train_counts=count_vect.fit_transform(X_train)
X_test_cv = count_vect.transform(X_test)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB(alpha=1).fit(X_train_tfidf, y_train)#alpha=1 ->Laplace Smoothing

predictions = clf.predict(X_test_cv)

print('Naive Bayes score: ',accuracy_score(y_test, predictions))

Naive Bayes score:  0.7373868046571799


# Scores of NaiveBayes,Random Forest,SVM with the use of biagrams and TF-IDF 

In [12]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  5 15:34:22 2020

@author: orestis
"""
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
import re

lemmatizer = WordNetLemmatizer() 
DataLocation = './data.csv'

first_df = pd.read_csv(DataLocation,header=None,names=['Comment','Class'])

comments_biagrams=list()
#Create a new list with the bigrams of the comments
for i in range(len(first_df['Comment'])):
    txt1 = first_df['Comment'][i]
    # initializing list  
    test_list = [txt1] 
      
    # using list comprehension + enumerate() + split() 
    # for Bigram formation 
    res = [(x, i.split()[j + 1]) for i in test_list  
           for j, x in enumerate(i.split()) if j < len(i.split()) - 1] 
    
    new_comment = ''
    for x in res:
        #Create a new comment with the lemmatized biagrams
        new_comment = new_comment + lemmatizer.lemmatize(x[0]) + lemmatizer.lemmatize(x[1]) + ' '
    comments_biagrams.append(new_comment)

#Create a new dataset with the biagrams
class_values = first_df['Class']
DataSet = list(zip(comments_biagrams,class_values))
df = pd.DataFrame(data=DataSet,columns=['Comment','Class'])

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1,2), stop_words='english')

features = tfidf.fit_transform(df.Comment).toarray()
labels = df.Class


X_train,X_test,y_train,y_test = train_test_split(df['Comment'],df['Class'], random_state=0)

count_vect = CountVectorizer()

X_train_counts=count_vect.fit_transform(X_train)
X_test_cv = count_vect.transform(X_test)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB(alpha=1).fit(X_train_tfidf, y_train)

predictions = clf.predict(X_test_cv)

print('Naive Bayes score: ',accuracy_score(y_test, predictions))

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])


cv_df.groupby('model_name').accuracy.mean()



Naive Bayes score:  0.7373868046571799


model_name
LinearSVC                 0.729534
RandomForestClassifier    0.656098
Name: accuracy, dtype: float64